In [1]:
from omegaconf import OmegaConf
import os
import torch

import hydra
from omegaconf import DictConfig, OmegaConf
from models.together_model import ProteinVAELLMmodel, ProteinVAELLM_FrameDiff_first_model
from data import all_atom
# Pytorch lightning imports
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.loggers.wandb import WandbLogger
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

from data.pdb_dataloader import PdbDataModule
from models.flow_module import FlowModule
from experiments import utils as eu
import openfold.utils.rigid_utils as ru
import torch.nn.functional as F
cfg = OmegaConf.load("configs/base.yaml")
_cfg = cfg
_data_cfg = cfg.data
_exp_cfg = cfg.experiment
_datamodule: LightningDataModule = PdbDataModule(_data_cfg)
_datamodule.setup(stage="fit")
torch.autograd.set_detect_anomaly(True)

/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import shutil
pdb_path = "/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06/length_100/sample_0/sample.pdb"
sc_output_dir = os.path.join("/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06/length_100/sample_0", 'self_consistency')
os.makedirs(sc_output_dir, exist_ok=True)
        

In [5]:
os.path.basename(pdb_path), os.path.join(sc_output_dir, os.path.basename(pdb_path))

('sample.pdb',
 '/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06/length_100/sample_0/self_consistency/sample.pdb')

In [6]:
shutil.copy(pdb_path, os.path.join(sc_output_dir, os.path.basename(pdb_path)))        

'/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06/length_100/sample_0/self_consistency/sample.pdb'

In [ ]:
_pmpnn_dir = 
seq_per_sample = 
gpu_id = 

def run_self_consistency(
        decoy_pdb_dir: str,
        reference_pdb_path: str,
        motif_mask: Optional[np.ndarray]=None):
    """Run self-consistency on design proteins against reference protein.
        
        Args:
            decoy_pdb_dir: directory where designed protein files are stored.
            reference_pdb_path: path to reference protein file
            motif_mask: Optional mask of which residues are the motif.

        Returns:
            Writes ProteinMPNN outputs to decoy_pdb_dir/seqs
            Writes ESMFold outputs to decoy_pdb_dir/esmf
            Writes results in decoy_pdb_dir/sc_results.csv
    """

    # Run PorteinMPNN
    output_path = os.path.join(decoy_pdb_dir, "parsed_pdbs.jsonl")
    process = subprocess.Popen([
        'python',
        f'{_pmpnn_dir}/helper_scripts/parse_multiple_chains.py',
        f'--input_path={decoy_pdb_dir}',
        f'--output_path={output_path}',
    ])
    _ = process.wait()
    num_tries = 0
    ret = -1
    pmpnn_args = [
        'python',
        f'{_pmpnn_dir}/protein_mpnn_run.py',
        '--out_folder',
        decoy_pdb_dir,
        '--jsonl_path',
        output_path,
        '--num_seq_per_target',
        str(seq_per_sample),
        '--sampling_temp',
        '0.1',
        '--seed',
        '38',
        '--batch_size',
        '1',
    ]
    if gpu_id is not None:
        pmpnn_args.append('--device')
        pmpnn_args.append(str(gpu_id))
    while ret < 0:
        try:
            process = subprocess.Popen(
                pmpnn_args,
                stdout=subprocess.DEVNULL,
                stderr=subprocess.STDOUT
            )
            ret = process.wait()
        except Exception as e:
            num_tries += 1
            print(f'Failed ProteinMPNN. Attempt {num_tries}/5')
            torch.cuda.empty_cache()
            if num_tries > 4:
                raise e
    mpnn_fasta_path = os.path.join(
        decoy_pdb_dir,
        'seqs',
        os.path.basename(reference_pdb_path).replace('.pdb', '.fa')
    )

        # Run ESMFold on each ProteinMPNN sequence and calculate metrics.
    mpnn_results = {
        'tm_score': [],
        'sample_path': [],
        'header': [],
        'sequence': [],
        'rmsd': [],
    }
    if motif_mask is not None:
        # Only calculate motif RMSD if mask is specified.
        mpnn_results['motif_rmsd'] = []
    esmf_dir = os.path.join(decoy_pdb_dir, 'esmf')
    os.makedirs(esmf_dir, exist_ok=True)
    fasta_seqs = fasta.FastaFile.read(mpnn_fasta_path)
    sample_feats = du.parse_pdb_feats('sample', reference_pdb_path)
    for i, (header, string) in enumerate(fasta_seqs.items()):

        # Run ESMFold
        esmf_sample_path = os.path.join(esmf_dir, f'sample_{i}.pdb')
        _ = run_folding(string, esmf_sample_path)
        esmf_feats = du.parse_pdb_feats('folded_sample', esmf_sample_path)
        sample_seq = du.aatype_to_seq(sample_feats['aatype'])

        # Calculate scTM of ESMFold outputs with reference protein
        _, tm_score = metrics.calc_tm_score(
            sample_feats['bb_positions'], esmf_feats['bb_positions'],
            sample_seq, sample_seq)
        rmsd = metrics.calc_aligned_rmsd(
            sample_feats['bb_positions'], esmf_feats['bb_positions'])
        if motif_mask is not None:
            sample_motif = sample_feats['bb_positions'][motif_mask]
            of_motif = esmf_feats['bb_positions'][motif_mask]
            motif_rmsd = metrics.calc_aligned_rmsd(
                sample_motif, of_motif)
            mpnn_results['motif_rmsd'].append(motif_rmsd)
        mpnn_results['rmsd'].append(rmsd)
        mpnn_results['tm_score'].append(tm_score)
        mpnn_results['sample_path'].append(esmf_sample_path)
        mpnn_results['header'].append(header)
        mpnn_results['sequence'].append(string)

        # Save results to CSV
    csv_path = os.path.join(decoy_pdb_dir, 'sc_results.csv')
    mpnn_results = pd.DataFrame(mpnn_results)
    mpnn_results.to_csv(csv_path)


def run_folding(self, sequence, save_path):
    """Run ESMFold on sequence."""
    with torch.no_grad():
        output = self._folding_model.infer_pdb(sequence)
    with open(save_path, "w") as f:
        f.write(output)
    return output